In [1]:
!pip install deepface
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.9 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=ed9dab037bd217d40550131418170605aede8ee13960fd7fa6daa2116988e23e
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 902.2/902.2 kB 30.8 MB/s eta 0:00:00


In [2]:
import numpy as np
import os
import cv2
import pandas as pd
import gdown
from deepface.models.face_detection import OpenCv
from deepface.models.face_detection import MtCnn
from deepface.models.face_detection import Yolo
from google.colab import drive
from google.colab.patches import cv2_imshow

In [3]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/dataset_biometric'
os.listdir(path)

Mounted at /content/drive


['Wed5a', 'PersonData.txt']

In [4]:
dataset_path = '/content/drive/MyDrive/dataset_biometric/Wed5a'

#OpenCv model
face_detector_opencv = OpenCv.OpenCvClient()

# Hàm xử lý hình ảnh
def detect_faces_in_image_opencv(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Không thể đọc hình ảnh: {image_path}")
        return None

    # Phát hiện khuôn mặt
    detections = face_detector_opencv.detect_faces(image)

    if detections is None:
        return []
    else:
        return detections

    # print(f"Đã phát hiện {len(detections)} khuôn mặt trong hình ảnh: {image_path}")

    # # Hiển thị kết quả
    # cv2_imshow(image)  # Sử dụng cv2_imshow để hiển thị ảnh trong Colab

# Duyệt qua tập dữ liệu và phát hiện khuôn mặt
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith('.jpg'):
            image_path = os.path.join(root, file)
            detect_faces_in_image_opencv(image_path)
print("successfully!")

successfully!


In [5]:
#MTCNN Model
face_detector_mtcnn = MtCnn.MtCnnClient()

def detect_faces_in_image_mtcnn(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Không thể đọc hình ảnh: {image_path}")
        return None

    # Phát hiện khuôn mặt
    detections = face_detector_mtcnn.detect_faces(image)

    return detections if detections is not None else []

    # print(f"Đã phát hiện {len(detections)} khuôn mặt trong hình ảnh: {image_path}")

    # cv2_imshow(image)

# Duyệt qua tập dữ liệu và phát hiện khuôn mặt
for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith('.jpg'):
            image_path = os.path.join(root, file)
            detect_faces_in_image_mtcnn(image_path)
print("successfully!")

successfully!


In [6]:
#Yolo model
os.makedirs('/root/.deepface/weights', exist_ok=True)

# Download the Yolo weights manually using gdown
weight_url = 'https://drive.google.com/uc?id=1qcr9DbgsX3ryrz2uU8w4Xm3cOrRywXqb'
weight_path = '/root/.deepface/weights/yolov8n-face.pt'
gdown.download(weight_url, weight_path, quiet=False)

face_detector_yolo = Yolo.YoloClient()

def detect_faces_in_image_yolo(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Không thể đọc hình ảnh: {image_path}")
        return None

    detections = face_detector_yolo.detect_faces(image)

    return detections if detections is not None else []
    # print(f"Đã phát hiện {len(detections)} khuôn mặt trong hình ảnh: {image_path}")

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.lower().endswith('.jpg'):
            image_path = os.path.join(root, file)
            detect_faces_in_image_yolo(image_path)
print("successfully!")

Downloading...
From: https://drive.google.com/uc?id=1qcr9DbgsX3ryrz2uU8w4Xm3cOrRywXqb
To: /root/.deepface/weights/yolov8n-face.pt
100%|██████████| 6.39M/6.39M [00:00<00:00, 32.7MB/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
successfully!


In [7]:
def load_ground_truth(file_path):
    """
    Đọc file .txt để tạo ground_truth.
    Args:
        file_path (str): Đường dẫn tới file ground_truth.txt.
    Returns:
        dict: Dictionary với key là tên ảnh và value là số khuôn mặt thực tế.
    """
    ground_truth = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 2:  # Kiểm tra dòng hợp lệ
                # image_name = parts[0]
                # face_count = int(parts[1])
                # ground_truth[image_name] = face_count
                filename, num_faces = parts
                ground_truth[filename] = int(num_faces)
    return ground_truth

In [8]:
import time
from sklearn.metrics import accuracy_score

dataset_path = "/content/drive/MyDrive/dataset_biometric/Wed5a"
ground_truth_file = "/content/drive/MyDrive/dataset_biometric/PersonData.txt"
ground_truth = load_ground_truth(ground_truth_file)

def evaluate_model(detect_faces_function, model_name, dataset_path, ground_truth):
    total_time = 0
    all_predictions = []
    all_labels = []

    print(f"Đang đánh giá mô hình {model_name}...")

    # Duyệt qua tập dữ liệu
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.jpg')):
                image_path = os.path.join(root, file)

                # Đọc nhãn thực tế từ ground_truth
                label = ground_truth.get(file, 0)  # Mặc định là 0 nếu không có nhãn

                # Bắt đầu phát hiện khuôn mặt
                start_time = time.time()
                try:
                    detections = detect_faces_function(image_path)
                    num_faces_detected = len(detections) if detections is not None else 0
                except Exception as e:
                    print(f"Lỗi khi xử lý {image_path}: {e}")
                    num_faces_detected = 0  # Nếu xảy ra lỗi, xem như phát hiện 0 khuôn mặt
                elapsed_time = time.time() - start_time

                # Tính tổng thời gian
                total_time += elapsed_time

                # Lưu kết quả dự đoán và nhãn thực tế
                all_predictions.append(num_faces_detected)
                all_labels.append(label)

                # In log cho từng ảnh
                print(f"[{model_name}] Ảnh: {file}, Nhãn thực tế: {label}, Phát hiện: {num_faces_detected}, "
                      f"Thời gian: {elapsed_time:.4f} giây")

    # Tính độ chính xác và thời gian trung bình
    accuracy = accuracy_score(all_labels, all_predictions)
    avg_time = total_time / len(all_predictions) if all_predictions else 0

    # Hiển thị kết quả tổng quan
    print(f"\nKết quả cho mô hình {model_name}:")
    print(f" - Độ chính xác: {accuracy:.2f}")
    print(f" - Thời gian trung bình mỗi ảnh: {avg_time:.4f} giây")
    print(f" - Tổng thời gian: {total_time:.4f} giây")
    print("\n")

    return accuracy, avg_time
# Đánh giá từng mô hình
print("Bắt đầu đánh giá...")
opencv_accuracy, opencv_time = evaluate_model(detect_faces_in_image_opencv, "OpenCV", dataset_path, ground_truth)
mtcnn_accuracy, mtcnn_time = evaluate_model(detect_faces_in_image_mtcnn, "MTCNN", dataset_path, ground_truth)
yolo_accuracy, yolo_time = evaluate_model(detect_faces_in_image_yolo, "YOLO", dataset_path, ground_truth)

# So sánh các mô hình
print("So sánh mô hình:")
print(f"OpenCV: Accuracy={opencv_accuracy:.2f}, Avg Time={opencv_time:.4f}s")
print(f"MTCNN: Accuracy={mtcnn_accuracy:.2f}, Avg Time={mtcnn_time:.4f}s")
print(f"YOLO: Accuracy={yolo_accuracy:.2f}, Avg Time={yolo_time:.4f}s")


Bắt đầu đánh giá...
Đang đánh giá mô hình OpenCV...
[OpenCV] Ảnh: 1157890395_1a59f09727_1238_83458989@N00.jpg, Nhãn thực tế: 6, Phát hiện: 7, Thời gian: 0.9090 giây
[OpenCV] Ảnh: 2165027619_ebb53085b0_2358_11246063@N07.jpg, Nhãn thực tế: 6, Phát hiện: 6, Thời gian: 0.2209 giây
[OpenCV] Ảnh: 2165027427_e6225271fb_2023_11246063@N07.jpg, Nhãn thực tế: 5, Phát hiện: 5, Thời gian: 0.2417 giây
[OpenCV] Ảnh: 2165026035_180be93896_2379_11246063@N07.jpg, Nhãn thực tế: 6, Phát hiện: 4, Thời gian: 0.4134 giây
[OpenCV] Ảnh: 2215543537_505e1b9637_2237_11246063@N07.jpg, Nhãn thực tế: 6, Phát hiện: 2, Thời gian: 0.2572 giây
[OpenCV] Ảnh: 1074907890_54830d807a_1136_8011736@N02.jpg, Nhãn thực tế: 7, Phát hiện: 2, Thời gian: 0.1897 giây
[OpenCV] Ảnh: 1158763262_f5a7990202_1347_83458989@N00.jpg, Nhãn thực tế: 6, Phát hiện: 6, Thời gian: 1.2851 giây
[OpenCV] Ảnh: 394412904_97332fba0a_149_84477621@N00.jpg, Nhãn thực tế: 6, Phát hiện: 6, Thời gian: 0.2132 giây
[OpenCV] Ảnh: 2229985817_3e3dc0dcc7_2232_161047